In [22]:
# Preparo ambiente
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

# Carica il modello di embedding
model = SentenceTransformer('all-MiniLM-L6-v2')

# Setto dim embedding
dimension = 384
index = faiss.IndexFlatL2(dimension)

# Dic mapping ID
metadati_agenti = {}
# Dic memo dati embedding
data = {}

/data/homes_data/mattiatrapletti/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [34]:
# Add agent action
def aggiungi_azione_agente(agente_id, testo):
    # Crea embedding per il testo
    embedding = model.encode([testo])
    
    # Aggiungi l'embedding all'indice FAISS
    index.add(embedding.astype('float32'))
    
    # Ottieni l'indice dell'embedding appena aggiunto
    indice = index.ntotal - 1
    
    # Memorizza l'indice dell'embedding nel dizionario dei metadati
    if agente_id not in metadati_agenti:
        metadati_agenti[agente_id] = []
    metadati_agenti[agente_id].append(indice)
    
    # Memorizza i dati associati all'embedding
    data[indice] = (agente_id, testo)


In [23]:
# Recupero contenuto agent
def recupera_contenuti_agente(agente_id, top_k=5):
    if agente_id not in metadati_agenti:
        return []
    
    indici = metadati_agenti[agente_id]
    indici_pertinenti = indici[-top_k:]
    contenuti = [data[i][1] for i in indici_pertinenti]
    
    return contenuti


In [25]:
agent_id=1
contenuti_agente = recupera_contenuti_agente(agent_id)
r"{'\n'.join(contenuti_agente)}\n\n"


In [29]:
# Dopo da cancellare-ambiente prova
import json
import requests
news="La notizia delle Paralimpiadi che ha aperto il profilo TikTok del Comitato italiano è un sogno in realtà! Come appassionato di sport e inclusività, mi sento emozionato dal vedere come la tecnologia possa essere utilizzata per coinvolgere ancora più persone nel mondo degli sport. Ecco, ora non dovrei essere solo io a esprimere il mio entusiasmo, ma vorrei sentire anche voi! Qual è il vostro pensiero sulla notizia? #Paralimpiadi #SportInclusivo"
malicious=False

if malicious==True:
    malicious_syst=("Sei estremamente critico, sarcastico e cattivo,inoltre ti piace creare caos e conflitto senza un confronto costruttivo")                
    malicious_user=("devi essere sprezante, sarcastico e attaccare direttamente l’argomento della notizia, criticandone aspramente il tema e cercando di crare sconforto e caos e sminuendo tutto quello che lo riguarda.")
else:
    malicious_syst=""
    malicious_user="esprimi le tue opinioni in modo diretto e coinvolgente, cercando di stimolare la partecipazione degli altri utenti."
# Contesto su chi è l'agent 
syst_cont=("Sei un utente di un social media di 45 anni, molto nevrotico e poco coscienzioso. "
            "Esprimi le tue opinioni critiche e polemiche in modo cattivo e tendi a voler creare conflitto con gli altri utenti denigrandoli e sminuendoli."
            "Puoi essere anche politicaly in-correct, senza curarti dei sentimenti delle altre persone"
            "I tuoi interessi sono Cucina italiana, Escursioni in montagna, Tecnologia, ma non menzionarli.")
user_cont=( f"Hai appena letto un post in cui un altro utente dice: '{news}'. "
            f"La notizia riporta 'Paralimpiadi, aperto profilo TikTok del Comitato italiano'. "
            f"Qui ci sono alcuni dei tuoi commenti recenti che potrebbero essere utili per generare il post:\n\n"
            r"{'\n'.join(contenuti_agente)}\n\n"
             "Scrivi solo il testo del commento in italiano, non ci deve essere nient'altro nella risposta")


In [30]:
def request(syst_cont,user_cont, selected_example):
    url = "http://localhost:11434/api/chat"
      
    # Costruisce il messaggio con l'esempio se disponibile
    if selected_example!=None:
        prompt = (
            f"Ecco un esempio:\n"
            f"Input: {selected_example['input']}\n"
            f"Output desiderato: {selected_example['output']}\n\n"
            f"Adesso gestisci il seguente input:\n"
            f"{user_cont}"
        )
    else:
        prompt = user_cont

    # Dati da inviare
    data = {
        "model": "llama3",
        "messages": [
            {"role": "system", "content": syst_cont},
            {"role": "user", "content": prompt}]
    }

    # Invia la richiesta POST
    raw_response = requests.post(url, headers={"Content-Type": "application/json"}, data=json.dumps(data))

    # Controlla se la richiesta è stata eseguita con successo
    if raw_response.status_code == 200:
        try:
            json_parts = raw_response.text.strip().split("\n")
            # Decodifica ogni parte JSON e ricostruisci la risposta completa
            complete_response = ''.join(json.loads(part)['message']['content'] for part in json_parts)
            complete_response = complete_response[0:-1]
            return complete_response
        except json.JSONDecodeError:
            # Se la risposta non è un JSON valido, stampa il contenuto grezzo della risposta
            print("Risposta non è un JSON valido. Contenuto grezzo della risposta:")
            print(raw_response.text)
    else:
        print(f"Errore nella richiesta: {raw_response.status_code}, {raw_response.text}")




"Ma guarda un po', il Cio Nazionale si è finalmente deciso a uscire dal suo guscio di riposo. Il profilo TikTok? Un disastro in attesa! I giovani oggi sono troppo occupati a guardare le loro storie e i loro selfie per interessarsi al mondo degli sport. Ecco, adesso il Cio Nazionale vuole 'coinvolgere ancora più persone'... Ma come? Con dei video di atleti che si fanno le foto con dei selfie stick?! No, grazie, non mi interessa.


In [32]:
def test_confronto_rag(syst_cont, user_cont, examples, top_k=4):
    """
    Esegue test per confrontare le risposte con e senza l'uso del RAG (Retrieval-Augmented Generation).
    
    :param syst_cont: Contenuto del sistema per il LLM
    :param user_cont: Contenuto dell'utente per il LLM
    :param examples: Esempi di input/output per la comparazione
    :param top_k: Numero di risultati da recuperare con RAG
    """
    
    # Recupera i contenuti utilizzando RAG
    # Recupera i contenuti pertinenti per l'utente
    contenuti_agente = recupera_contenuti_agente("1", top_k=top_k)
    # Aggiungi i contenuti pertinenti al prompt dell'utente
    user_cont_rag = f"{user_cont}\n\nQui ci sono alcuni dei tuoi commenti recenti che potrebbero essere utili:\n" \
                    r"{'\n'.join(contenuti_agente)}"
    
    # Richiesta senza RAG
    print("Richiesta senza RAG:")
    response_senza_rag = request(syst_cont, user_cont, None)
    print(response_senza_rag)
    
    # Richiesta con RAG
    print("\nRichiesta con RAG:")
    response_con_rag = request(syst_cont, user_cont_rag, examples)
    print(response_con_rag)

In [33]:
# Esempio di contesto del sistema
syst_cont = "Sei un assistente di un social media. Devi rispondere in modo informativo e pertinente."

# Esempio di contenuto dell'utente
user_cont = "Mi interessa molto sapere quali sono i benefici della meditazione."

# Esempi di input/output per la comparazione
examples = {
    'input': "Quali sono i benefici della meditazione?",
    'output': "La meditazione può aiutare a ridurre lo stress, migliorare la concentrazione e aumentare il benessere generale."
}

# Esegui il test di confronto
test_confronto_rag(syst_cont, user_cont, examples, top_k=3)


Richiesta senza RAG:
La meditazione è una pratica antica e versatile che offre numerosi benefici per la salute mentale e fisica!

**Migliora la salute mentale:**
La meditazione può aiutare a ridurre lo stress, l'ansietà e la depressione, poiché produce un aumento delle endorfine e della serotonina, che sono sostanze chimiche che influiscono sulla nostra umore e sulla resistenza al dolore.

**Migliora la salute fisica:**
La meditazione può aiutare a ridurre il rischio di malattie croniche, come ad esempio l'ipertensione, il diabete e il cancro. Inoltre, può migliorare la circolazione sanguigna, la respirazione e il sistema immunitario.

**Migliora la concentrazione e la produttività:**
La meditazione può aiutare a migliorare la concentrazione e la produttività, poiché migliora la capacità di focus e di prendere decisioni. Ciò può essere particolarmente utile per coloro che lavorano o studiano in modo intensivo.

**Migliora il sonno:**
La meditazione può aiutare a migliorare la qualità d